# MVP Award

In [242]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

awards_data = pd.read_csv("../../initial_data/awards_players.csv")
teams_data = pd.read_csv("../../initial_data/teams.csv")
players_teams_clean = pd.read_csv("../awards_data/players_teams_clean.csv")

In [243]:
# Check the column names in teams_data
print("Teams data columns:")
print(teams_data.columns.tolist())
print("\nPlayers teams clean columns:")
print(players_teams_clean.columns.tolist())
print("\nTeams data sample:")
print(teams_data.head())

Teams data columns:
['year', 'lgID', 'tmID', 'franchID', 'confID', 'divID', 'rank', 'playoff', 'seeded', 'firstRound', 'semis', 'finals', 'name', 'o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB', 'won', 'lost', 'GP', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'min', 'attend', 'arena']

Players teams clean columns:
['playerID', 'year', 'team', 'minutes', 'games_played', 'total_points', 'total_rebounds', 'total_assists', 'points_per_min', 'assists_per_min', 'rebounds_per_min', 'steals_per_min', 'blocks_per_min', 'turnovers_per_min', 'FG%', 'FT%', 'Three%', 'Three Rate']

Teams data sample:
   year  lgID tmID franchID confID  divID  rank playoff  seeded firstRound  \
0     9  WNBA  ATL      ATL    

## Create MVP Data

In [244]:
# Join dataset players_teams_clean data with teams_data (team_wins, team_losses, rank, playoff_made)
# Note: players_teams_clean has 'team' column, teams_data has 'tmID' column
mvp_data = players_teams_clean.merge(
    teams_data[['year', 'tmID', 'won', 'lost', 'rank', 'playoff', 'confID']], 
    left_on=['year', 'team'], 
    right_on=['year', 'tmID'], 
    how='left'
)

# Rename columns for clarity
mvp_data.rename(columns={
    'won': 'team_wins', 
    'lost': 'team_losses', 
    'rank': 'conference_rank', 
    'playoff': 'playoff_made'
}, inplace=True)

# Drop the duplicate tmID column (we already have 'team')
mvp_data.drop(columns=['tmID'], inplace=True)

# Change Playoff from N / Y to 0 / 1
mvp_data['playoff_made'] = mvp_data['playoff_made'].map({'Y': 1, 'N': 0})
# Change ConfID to numeric  EA / WE to 0 / 1
mvp_data['confID'] = mvp_data['confID'].astype('category').cat.codes

# Add MVP winner field (1 if player won MVP that year, 0 otherwise)
mvp_winners = awards_data[awards_data['award'] == 'Most Valuable Player'][['year', 'playerID']]
mvp_data['MVP_winner'] = mvp_data.apply(
    lambda row: 1 if ((mvp_winners['year'] == row['year']) & 
                      (mvp_winners['playerID'] == row['playerID'])).any() else 0,
    axis=1
)

print(f"MVP data shape: {mvp_data.shape}")
print(f"\nColumns: {mvp_data.columns.tolist()}")
print(f"\nMVP winners in dataset: {mvp_data['MVP_winner'].sum()}")
print(f"\nSample data:")
print(mvp_data.head())
print(f"\nMVP winners:")
print(mvp_data[mvp_data['MVP_winner'] == 1][['playerID', 'year', 'team', 'total_points', 'games_played']])

# Save csv (relative path from current location)
mvp_data.to_csv("mvp_data.csv", index=False)
print(f"\n✅ Saved to mvp_data.csv")


MVP data shape: (1876, 24)

Columns: ['playerID', 'year', 'team', 'minutes', 'games_played', 'total_points', 'total_rebounds', 'total_assists', 'points_per_min', 'assists_per_min', 'rebounds_per_min', 'steals_per_min', 'blocks_per_min', 'turnovers_per_min', 'FG%', 'FT%', 'Three%', 'Three Rate', 'team_wins', 'team_losses', 'conference_rank', 'playoff_made', 'confID', 'MVP_winner']

MVP winners in dataset: 10

Sample data:
     playerID  year team  minutes  games_played  total_points  total_rebounds  \
0  abrossv01w     2  MIN      846            26           343             174   
1  abrossv01w     3  MIN      805            27           314             146   
2  abrossv01w     4  MIN      792            30           318             141   
3  abrossv01w     5  MIN      462            22           146              74   
4  abrossv01w     6  MIN      777            31           304             107   

   total_assists  points_per_min  assists_per_min  ...       FG%       FT%  \
0         

## Define features to use in the models

In [245]:
# Define feature columns for the model
# Using year N-1 stats to predict year N MVP
feature_columns = [
    'minutes','games_played','total_points','total_rebounds','total_assists',
    'points_per_min','assists_per_min','rebounds_per_min','steals_per_min',
    'blocks_per_min','turnovers_per_min','FG%','FT%','Three%','Three Rate',
    'team_wins','team_losses','conference_rank','playoff_made','confID'
]

print("Features for MVP prediction:")
for i, col in enumerate(feature_columns, 1):
    print(f"{i}. {col}")


Features for MVP prediction:
1. minutes
2. games_played
3. total_points
4. total_rebounds
5. total_assists
6. points_per_min
7. assists_per_min
8. rebounds_per_min
9. steals_per_min
10. blocks_per_min
11. turnovers_per_min
12. FG%
13. FT%
14. Three%
15. Three Rate
16. team_wins
17. team_losses
18. conference_rank
19. playoff_made
20. confID


## Prediction Models

### 1. Logistic Regression

In [246]:
# Load data and prepare for training
# Strategy: Use year N-1 stats to predict year N MVP
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

mvp_data = pd.read_csv("mvp_data.csv")

# Create training dataset: year N-1 stats → year N MVP
# For each player's stats in year N-1, check if they won MVP in year N
mvp_data['next_year'] = mvp_data['year'] + 1

# Get MVP winners for next year
mvp_next_year = mvp_data[mvp_data['MVP_winner'] == 1][['year', 'playerID']].copy()
mvp_next_year.rename(columns={'year': 'next_year'}, inplace=True)

# Merge to create target variable
mvp_data = mvp_data.merge(
    mvp_next_year,
    on=['next_year', 'playerID'],
    how='left',
    indicator=True
)
mvp_data['MVP_winner_next_year'] = (mvp_data['_merge'] == 'both').astype(int)
mvp_data.drop(columns=['_merge'], inplace=True)

# Filter to MVP candidates only
mvp_candidates = mvp_data[
    (mvp_data['minutes'] >= 500) &
    (mvp_data['games_played'] >= 20)
].copy()

print(f"Total candidates: {len(mvp_candidates)}")
print(f"MVP winners (next year): {mvp_candidates['MVP_winner_next_year'].sum()}")
print(f"Years: {sorted(mvp_candidates['year'].unique())}")


Total candidates: 866
MVP winners (next year): 7
Years: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


#### Training the model

In [247]:
# Train Logistic Regression Model

# EXAMPLE: To predict year 7 -> exclude year 6 from training (add it to testing), use all other years
train_years = [1, 2, 3, 4, 5, 6, 7, 8]
test_years = [9]

train_data = mvp_candidates[mvp_candidates['year'].isin(train_years)].copy()
test_data = mvp_candidates[mvp_candidates['year'].isin(test_years)].copy()

X_train = train_data[feature_columns]
y_train = train_data['MVP_winner_next_year']
X_test = test_data[feature_columns]
y_test = test_data['MVP_winner_next_year']

print(f"Training: {len(X_train)} candidates, {y_train.sum()} MVP winners")
print(f"Testing: {len(X_test)} candidates, {y_test.sum()} MVP winners")

# Scale and train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

test_data['mvp_probability'] = y_pred_proba
test_data['mvp_prediction'] = y_pred

print(f"\n✅ Model trained!")
print(f"Train accuracy: {model.score(X_train_scaled, y_train):.3f}")
print(f"Test accuracy: {model.score(X_test_scaled, y_test):.3f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Not MVP', 'MVP']))


Training: 690 candidates, 6 MVP winners
Testing: 89 candidates, 1 MVP winners

✅ Model trained!
Train accuracy: 0.946
Test accuracy: 0.910

Classification Report:
              precision    recall  f1-score   support

     Not MVP       1.00      0.91      0.95        88
         MVP       0.11      1.00      0.20         1

    accuracy                           0.91        89
   macro avg       0.56      0.95      0.58        89
weighted avg       0.99      0.91      0.94        89



In [248]:
# Show test results
test_results = test_data.sort_values('mvp_probability', ascending=False)
print("\nYear 9 Stats → Year 10 MVP Predictions:")
print(f"{'Rank':<6}{'Player':<25}{'Probability':<12}{'Actual MVP'}")
print("-" * 60)
for idx, (_, row) in enumerate(test_results.head(10).iterrows(), 1):
    is_mvp = "✅" if row['MVP_winner_next_year'] == 1 else ""
    print(f"{idx:<6}{row['playerID']:<25}{row['mvp_probability']:.4f}{'':6}{is_mvp}")



Year 9 Stats → Year 10 MVP Predictions:
Rank  Player                   Probability Actual MVP
------------------------------------------------------------
1     parkeca01w               0.9900      
2     tauradi01w               0.9772      ✅
3     leslili01w               0.9553      
4     anosini01w               0.9313      
5     augusse01w               0.9065      
6     jacksla01w               0.8844      
7     smithta01w               0.7218      
8     pondeca01w               0.7061      
9     youngso01w               0.5281      
10    wiggica01w               0.2921      


In [249]:
# Diagnostic: Check feature importance and actual MVP stats
print("\n🔍 Feature Importance (Top 10):")
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'coefficient': model.coef_[0]
}).sort_values('coefficient', ascending=False)
print(feature_importance.head(10))

# Check if actual MVP is in the test data
# Get the predicted year from test_years variable
predicted_year = test_years[0] + 1  # test_years contains stats year, add 1 for predicted year
stats_year = test_years[0]

print(f"\n🔍 Actual Year {predicted_year} MVP in test data:")
actual_mvp = test_data[test_data['MVP_winner_next_year'] == 1]
if len(actual_mvp) > 0:
    print(f"MVP: {actual_mvp.iloc[0]['playerID']}")
    print(f"Probability: {actual_mvp.iloc[0]['mvp_probability']:.4f}")
    print(f"Year {stats_year} stats:")
    print(actual_mvp[['total_points', 'total_rebounds', 'total_assists', 'games_played', 'team_wins', 'conference_rank']].iloc[0])
else:
    print("❌ MVP not found in test candidates!")
    print(f"Checking if MVP exists in year {stats_year} data at all...")
    predicted_year_mvp = mvp_data[(mvp_data['year'] == predicted_year) & (mvp_data['MVP_winner'] == 1)]
    if len(predicted_year_mvp) > 0:
        print(f"Year {predicted_year} MVP: {predicted_year_mvp.iloc[0]['playerID']}")
        stats_year_data = mvp_data[(mvp_data['year'] == stats_year) & (mvp_data['playerID'] == predicted_year_mvp.iloc[0]['playerID'])]
        if len(stats_year_data) > 0:
            print(f"\nTheir Year {stats_year} stats:")
            print(stats_year_data[['minutes', 'games_played', 'total_points', 'team_wins', 'team_losses']].iloc[0])
            print(f"\nFiltered out because:")
            if stats_year_data.iloc[0]['minutes'] < 500:
                print(f"  ❌ Minutes: {stats_year_data.iloc[0]['minutes']} < 500")
            if stats_year_data.iloc[0]['games_played'] < 20:
                print(f"  ❌ Games: {stats_year_data.iloc[0]['games_played']} < 20")
        else:
            print(f"❌ MVP has no Year {stats_year} stats!")



🔍 Feature Importance (Top 10):
            feature  coefficient
19           confID     2.339543
9    blocks_per_min     1.626054
0           minutes     1.477143
5    points_per_min     1.245935
4     total_assists     1.086160
17  conference_rank     1.054944
15        team_wins     0.954452
3    total_rebounds     0.700812
16      team_losses     0.621534
8    steals_per_min     0.344431

🔍 Actual Year 10 MVP in test data:
MVP: tauradi01w
Probability: 0.9772
Year 9 stats:
total_points       820
total_rebounds     172
total_assists      121
games_played        34
team_wins           16
conference_rank      6
Name: 1589, dtype: int64


## Predict MVP for New Year

In [250]:
# Predict MVP for new year using previous year stats
# Example: year_10_players_teams_test.csv → use year 9 stats → predict year 10 MVP

new_year_file = "../../yearly_data/year_10_players_teams_test.csv"
predict_year = 10
stats_year = predict_year - 1

print(f"Predicting Year {predict_year} MVP using Year {stats_year} stats")

# Load new year player list
new_year_players = pd.read_csv(new_year_file)
print(f"Players in year {predict_year}: {len(new_year_players)}")

# Get year N-1 stats for these players
player_stats = mvp_data[
    (mvp_data['year'] == stats_year) &
    (mvp_data['playerID'].isin(new_year_players['playerID']))
].copy()

# Filter to MVP candidates
candidates = player_stats[
    (player_stats['minutes'] >= 500) &
    (player_stats['games_played'] >= 20)
].copy()

print(f"MVP candidates: {len(candidates)}")

# Predict
X_new = candidates[feature_columns]
X_new_scaled = scaler.transform(X_new)
candidates['mvp_probability'] = model.predict_proba(X_new_scaled)[:, 1]
candidates = candidates.sort_values('mvp_probability', ascending=False)

# Show top predictions
print(f"\nYear {predict_year} MVP Predictions (based on Year {stats_year} stats):")
print(f"{'Rank':<6}{'Player':<25}{'Team':<6}{'Probability':<12}{'PPG':<8}{'Wins'}")
print("-" * 70)
for idx, (_, row) in enumerate(candidates.head(15).iterrows(), 1):
    ppg = row['total_points'] / row['games_played']
    print(f"{idx:<6}{row['playerID']:<25}{row['team']:<6}{row['mvp_probability']:.4f}{'':6}{ppg:.1f}{'':6}{int(row['team_wins'])}")

print(f"\n🏆 Predicted MVP: {candidates.iloc[0]['playerID']}")

# Get actual MVP and their probability
actual_mvp_id = mvp_data[(mvp_data['year'] == predict_year) & (mvp_data['MVP_winner'] == 1)]['playerID'].values[0]
actual_mvp_prob = candidates[candidates['playerID'] == actual_mvp_id]['mvp_probability']
if len(actual_mvp_prob) > 0:
    print(f"✅ Actual MVP: {actual_mvp_id} // Calculated Probability: {actual_mvp_prob.values[0]:.4f}")
else:
    print(f"✅ Actual MVP: {actual_mvp_id} // Not in candidates (filtered out)")


Predicting Year 10 MVP using Year 9 stats
Players in year 10: 165
MVP candidates: 87

Year 10 MVP Predictions (based on Year 9 stats):
Rank  Player                   Team  Probability PPG     Wins
----------------------------------------------------------------------
1     parkeca01w               LAS   0.9900      18.5      20
2     tauradi01w               PHO   0.9772      24.1      16
3     leslili01w               LAS   0.9553      15.1      20
4     anosini01w               MIN   0.9313      9.2      16
5     augusse01w               MIN   0.9065      19.1      16
6     jacksla01w               SEA   0.8844      20.2      22
7     smithta01w               PHO   0.7218      11.1      16
8     pondeca01w               PHO   0.7061      21.2      16
9     youngso01w               SAS   0.5281      17.5      24
10    wiggica01w               MIN   0.2921      15.7      16
11    cashsw01w                SEA   0.2139      11.3      22
12    thompti01w               HOU   0.1056      18